In [462]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
#from keras import layers
#from keras import models

from music21 import *
import pickle
from gen_data import *

In [463]:
X_train = pickle.load(open("../data_train.txt",'rb'))
y_train = pickle.load(open("../labels_train.txt", 'rb'))

"""
X_train = np.concatenate((X_train[y_train == 0], X_train[y_train == 1])) 
y_train = np.concatenate((y_train[y_train == 0], y_train[y_train == 1])) 
"""
X_val = pickle.load(open("../data_val.txt", 'rb'))
y_val = pickle.load(open("../labels_val.txt", 'rb'))

"""
X_val = np.concatenate((X_val[y_val == 0], X_val[y_val == 1])) 
y_val = np.concatenate((y_val[y_val == 0], y_val[y_val == 1])) 
"""
means = pickle.load(open("../means.txt", 'rb'))
stds  = pickle.load(open("../stds.txt", 'rb'))
extremes = pickle.load(open("../extremes.txt", 'rb'))

print(X_train.shape)

(5507, 12, 20, 3)


In [464]:
H, W, C = X_train.shape[1:]
print(np.bincount(y_train))
print(np.bincount(y_val))
print(y_val[0], y_val[-1])

[ 510 1142  572 2986  297]
[  8 547 104 321 157]
2 4


In [465]:
print(X_train[20])

[[[-1.17640530e+00  4.96555358e-02 -5.56200055e-01]
  [-5.87647428e-01  6.16660875e-01 -5.46975664e-01]
  [-1.40224367e-01  6.12811429e-01 -6.64345820e-04]
  [-1.31267251e-01  6.05410631e-01  3.71808818e-02]
  [ 1.52116361e+00 -2.68027167e+00  1.27098444e+00]
  [ 1.54573714e+00 -2.62394887e+00  9.62056775e-02]
  [ 1.55628012e+00 -2.70549974e+00 -5.29633336e-01]
  [ 8.34609666e-01 -9.56959704e-02 -5.13047698e-01]
  [ 1.27570390e+00 -6.70426771e-02 -5.04262665e-01]
  [-1.42724012e+00 -7.82144767e-02 -5.13296015e-01]
  [-1.12710017e+00 -9.44412619e-02 -5.14237152e-01]
  [ 1.26478889e+00 -7.78360383e-02 -5.08529466e-01]
  [-1.46990178e+00 -7.20812981e-02 -5.10014641e-01]
  [ 1.24585842e+00 -7.64400406e-02 -8.20739128e-01]
  [ 9.28511243e-01 -7.38230119e-02 -8.28871838e-01]
  [ 1.23103445e+00 -3.61133005e-02 -5.00120613e-01]
  [-1.16356188e+00 -5.22578321e-02 -5.21552336e-01]
  [-1.13466006e+00 -7.42537901e-02 -5.08355323e-01]
  [-1.43422334e+00 -4.21359604e-02  1.31572345e-01]
  [ 1.249313

In [466]:
# Set up some global variables
USE_GPU = True

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 40

print('Using device: ', device)

Using device:  /device:GPU:0


In [467]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
# test_dset = Dataset(X_test, y_test, batch_size=64)

In [468]:
def ret_true():
    return True

def ret_false():
    return False


In [469]:
def check_accuracy(sess, dset, x, scores, is_training=None):
    num_correct, num_samples = 0, 0
    for x_batch, y_batch in dset:
        feed_dict = {x: x_batch, is_training: 0}
        scores_np = sess.run(scores, feed_dict=feed_dict)
        y_pred = scores_np.argmax(axis=1)
        num_samples += x_batch.shape[0]
        num_correct += (y_pred == y_batch).sum()
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))
    return acc

In [470]:
def check_accuracy_train(sess, x, scores, is_training=None):
    num_correct, num_samples = 0, 0
    
    feed_dict = {x: X_train, is_training: 0}
    scores_np = sess.run(scores, feed_dict=feed_dict)
    y_pred = scores_np.argmax(axis=1)
    num_samples += X_train.shape[0]
    num_correct += (y_pred == y_train).sum()
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

In [471]:
hidden1, hidden2, channel_1, channel_2, channel_3, channel_4, channel_5, channel_6, num_classes = 80, 250, 32, 64, 128, 256, 350, 512, 5

class MuseConvNet(tf.keras.Model):
    def layer_norm(self, x, scope, is_training):
        """
        Returns a batch normalization layer that automatically switch between train and test phases based on the 
        tensor is_training

        Args:
            x: input tensor
            scope: scope name
            is_training: boolean tensor or variable
            epsilon: epsilon parameter - see batch_norm_layer
            decay: epsilon parameter - see batch_norm_layer

        Returns:
            The correct batch normalization layer based on the value of is_training
        """
        #assert isinstance(is_training, (ops.Tensor, variables.Variable)) and is_training.dtype == tf.bool

        return tf.cond(
            is_training,
            lambda: self.layer_norm_layer(x=x, scope=scope, is_training=True, reuse=None),
            lambda: self.layer_norm_layer(x=x, scope=scope, is_training=False, reuse=True),
        )


    def layer_norm_layer(self, x, scope, is_training, reuse=None):
        return tf.contrib.layers.layer_norm(x, reuse=reuse, scope=scope, trainable=is_training)

    def __init__(self):
        super().__init__()
        
        initializer = tf.variance_scaling_initializer(scale=2.0)
        
        self.conv1 = tf.layers.Conv2D(channel_1, (8, 7), 1, "same", activation=tf.nn.relu, 
                                      use_bias=True, kernel_initializer=initializer, name="conv1")
        self.conv2 = tf.layers.Conv2D(channel_2, (6, 8), 1, "same", activation=tf.nn.relu, 
                                      use_bias=True, kernel_initializer=initializer, name="conv2")
        
        self.conv3 = tf.layers.Conv2D(channel_3, (5, 5), 1, "same", activation=tf.nn.relu, 
                                      use_bias=True, kernel_initializer=initializer, name="conv3")
        
        self.conv4 = tf.layers.Conv2D(channel_4, (6, 4), 1, "same", activation=tf.nn.relu, 
                                      use_bias=True, kernel_initializer=initializer, name="conv4")
        
        self.conv5 = tf.layers.Conv2D(channel_5, (4, 4), 1, "same", activation=tf.nn.relu, 
                                      use_bias=True, kernel_initializer=initializer, name="conv5")
        

        self.conv6 = tf.layers.Conv2D(channel_6, (4, 4), 1, "same", activation=tf.nn.relu, 
                                      use_bias=True, kernel_initializer=initializer, name="conv6") 
        
        
       
        self.fc1 = tf.layers.Dense(hidden1, use_bias = True,
                                  kernel_initializer=initializer, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.5), name="affine1")
        
        self.fc2 = tf.layers.Dense(hidden2, use_bias = True,
                                  kernel_initializer=initializer, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.5), name="affine2")
        
        self.fc = tf.layers.Dense(num_classes, use_bias = True,
                                  kernel_initializer=initializer, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.5), name="affine3")
        
    
    # Define model
    
    def call(self, inputs, is_training):
        X = inputs
        
        X = self.conv1(X)
        X = self.layer_norm(X, "batch1", is_training)
        
        X = self.conv2(X)
        X = self.layer_norm(X, "batch2", is_training)
        
        X = self.conv3(X)
        X = self.layer_norm(X, "batch3", is_training)
        
        
        
        X = self.conv4(X)
        X = self.layer_norm(X, "batch4", is_training)
        
        
        
        X = self.conv5(X)
        X = self.layer_norm(X, "batch5", is_training)
        
        
        X = self.conv6(X)
        X = self.layer_norm(X, "batch6", is_training)
        
        """
        X = self.conv7(X)
        X = tf.layers.batch_normalization(X, training=is_training, name="batch7")
        """
        X = tf.layers.flatten(X)
        
        X = self.fc1(X)
        """
        X = self.fc2(X)
        """
        scores = self.fc(X)
        
        return scores


In [472]:
best_sess = None
losses = []
def train(optimizer_init_fn, num_epochs=1):
    
    tf.reset_default_graph()    
    with tf.device(device):
        x = tf.placeholder(tf.float32, [None, H, W, C])
        y = tf.placeholder(tf.int32, [None])
        
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        # Use the model function to build the forward pass.
        model = MuseConvNet()
        scores = model(x, is_training)
        
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=scores)
        loss = tf.reduce_mean(loss)
        
        
        optimizer = optimizer_init_fn()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss)

    config = tf.ConfigProto(allow_soft_placement = True)
    saver = tf.train.Saver(tf.global_variables())
    with tf.Session(config=config) as sess:
        max_acc = 0
        sess.run(tf.global_variables_initializer())
        t = 0
        for epoch in range(num_epochs):
            print('Starting epoch %d' % epoch)
            for x_np, y_np in train_dset:
                if x_np.shape[0] != y_np.shape[0]:
                    print("Oh dear")
                feed_dict = {x: x_np, y: y_np, is_training:True}
                loss_np, _ = sess.run([loss, train_op], feed_dict=feed_dict)
                if t % print_every == 0:
                    print('Iteration %d, loss = %.4f' % (t, loss_np))
                    #print('Training accuracy')
                    #check_accuracy(sess, train_dset, x, scores, is_training=is_training)
                    print('Validation Accuracy')
                    acc = check_accuracy(sess, val_dset, x, scores, is_training=is_training)
                    """
                    if (acc > max_acc):
                        max_acc = acc
                        best_sess = sess
                    """
                    losses.append(loss_np)
                    print()
                t += 1
                
        print('Training accuracy final')
        acc = check_accuracy(sess, train_dset, x, scores, is_training=is_training)
        print('Validation accuracy final')
        acc = check_accuracy(sess, val_dset, x, scores, is_training=is_training)
        save_path = saver.save(sess, "model_test/model.ckpt")
        
        

In [473]:
learning_rate = 1e-5


def optimizer_init_fn():
    optimizer = None
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True)
    
    return optimizer

train(optimizer_init_fn, num_epochs=3)

Starting epoch 0
Iteration 0, loss = 2.1218
Validation Accuracy
Got 161 / 1137 correct (14.16%)

Iteration 40, loss = 0.0000
Validation Accuracy
Got 321 / 1137 correct (28.23%)

Iteration 80, loss = 0.0000
Validation Accuracy
Got 791 / 1137 correct (69.57%)

Starting epoch 1
Iteration 120, loss = 0.0015
Validation Accuracy
Got 386 / 1137 correct (33.95%)

Iteration 160, loss = 0.7693
Validation Accuracy
Got 839 / 1137 correct (73.79%)

Starting epoch 2
Iteration 200, loss = 0.2574
Validation Accuracy
Got 490 / 1137 correct (43.10%)

Iteration 240, loss = 5.9555
Validation Accuracy
Got 378 / 1137 correct (33.25%)

Training accuracy final
Got 4237 / 5507 correct (76.94%)
Validation accuracy final
Got 865 / 1137 correct (76.08%)
